In [9]:
import os
import sys
from time import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import random

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
import copy

from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')

from PatchyConverter import PatchyConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier
from ConvNetPatchy import ConvNetPatchy


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# New fucntions:
def print_nodes(nx_graph):
    for i in nx_graph.nodes:
        print(i,nx_graph.node[i])
def print_dict(dictio):
    for i in dictio.items():
        print(i)
        
    

### Load data

In [3]:
# Getting the data:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
s = 1
relabeling = True

# Get features:
graph_converter = PatchyConverter(dataset_name,receptive_field,s)
graph_tensor = graph_converter.graphs_to_Patchy_tensor()

# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)


importing graph data
number of graphs in MUTAG dataset : 188
number of features : 7
Separating Graphs per graph ID
getting width
width: 18
MUTAG tensor exists, loading it from Dropbox
Loading path: /Users/marcelogutierrez/.gamma_link/Samples/MUTAG/MUTAG_patchy_tensor_w_18.npy


In [4]:
mutag_tensor = graph_converter.process_data()

Time preprocess data in s 13.646012306213379


In [8]:
mutag_tensor2 = mutag_tensor.reshape(graph_converter.num_graphs,graph_converter.width, graph_converter.k,graph_converter.num_features)

In [13]:
val_acc = []
training_time = []
n_folds = 10
epochs = 400
for j in range(n_folds):
    # Starting Conv net with Patchy
    patchy_cnn = ConvNetPatchy(mutag_tensor2, graph_labels)
    patchy_cnn.split_data(random_state=j)
    patchy_cnn.build_graph()
    patchy_cnn.train_model(epochs=epochs)
    print('Fold {} completed with val acc : {}'.format(j, patchy_cnn.final_val_acc))

    val_acc.append(patchy_cnn.final_val_acc)
    training_time.append(patchy_cnn.training_time)



Fold 0 completed with val acc : 0.7894737124443054
Fold 1 completed with val acc : 1.0
Fold 2 completed with val acc : 0.7894737124443054
Fold 3 completed with val acc : 0.7368420958518982
Fold 4 completed with val acc : 0.9473684430122375
Fold 5 completed with val acc : 0.8947368264198303
Fold 6 completed with val acc : 0.9473684430122375
Fold 7 completed with val acc : 0.8421052694320679
Fold 8 completed with val acc : 0.7894737124443054
Fold 9 completed with val acc : 0.9473684430122375


In [ ]:
print(np.mean(val_acc))
print(np.std(val_acc))

0.8684210658073426

In [20]:
n_class = len(np.unique(graph_labels))

# Capsule Architecture Parameters:
capsule_params = CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params, conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2, 2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params, caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params, digit_layer_name)

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256  # 250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params, decoder_layer)

# Training Hyperparameters:

args_train = CapsuleTrainingParameters()
args_train.batch_size = 50


# Generate list of parameter sets::
list_parameter_sets = []
#list_parameter_sets.append(args_train)


lr = 0.001  
lr_decay = 0.9  
lam_recon = 0.392

training_params = CapsuleTrainingParameters(epochs=epochs,lr=lr,lr_decay=lr_decay,lam_recon=lam_recon)



fold_set = []

for j in range(n_folds):
    fold_set.append(train_test_split(mutag_tensor2,
                                     graph_labels,
                                     test_size=0.10,
                                     random_state=j))

results_df = []
val_acc = []
training_time=[]

for j in range(n_folds):

    x_train, x_test, y_train, y_test = fold_set[j]
    data = ((x_train, y_train), (x_test, y_test))
    input_shape = x_train.shape[1:]


    patchy_classifier = GraphClassifier(input_shape,n_class)
    patchy_classifier.build_the_graph(capsule_params)
    ##
    #patchy_classifier.train_model.summary()
    training_params.add_fold(j)
    training_params.verbose = 0
    ##
    patchy_classifier.train(data, training_params)

    training_time.append(patchy_classifier.training_time)
    val_acc.append(patchy_classifier.results.val_capsnet_acc)

    #if i == 0:
    results_df.append(pd.DataFrame(patchy_classifier.results))
    #else:
    print('Fold number {} trained '.format(j + 1))



time to generate the model: 0.3308100700378418
Trained model saved to './result/trained_model.h5'
Fold number 1 trained 


/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


time to generate the model: 0.29631805419921875
Trained model saved to './result/trained_model.h5'
Fold number 2 trained 
time to generate the model: 0.28438615798950195
Trained model saved to './result/trained_model.h5'
Fold number 3 trained 
time to generate the model: 0.3378407955169678
Trained model saved to './result/trained_model.h5'
Fold number 4 trained 
time to generate the model: 0.3094780445098877
Trained model saved to './result/trained_model.h5'
Fold number 5 trained 
time to generate the model: 0.2848930358886719
Trained model saved to './result/trained_model.h5'
Fold number 6 trained 
time to generate the model: 0.27689290046691895
Trained model saved to './result/trained_model.h5'
Fold number 7 trained 
time to generate the model: 0.276871919631958
Trained model saved to './result/trained_model.h5'
Fold number 8 trained 
time to generate the model: 0.27601003646850586
Trained model saved to './result/trained_model.h5'
Fold number 9 trained 
time to generate the model: 0

In [31]:
print(np.mean(pd.concat(results_df,1).loc['val_capsnet_acc',:]))
print(np.std(pd.concat(results_df,1).loc['val_capsnet_acc',:]))

0.8789473652839661
0.07080854598393455


## Color refinement

In [ ]:
graph_list = nx_graphs

In [ ]:
h=1
node_label=True
n = len(nx_graphs)
lists = [0] * n
k = [0] * (h + 1)
n_nodes = 0
n_max = 0

# Compute adjacency lists and n_nodes, the total number of
# nodes in the dataset.
for i in range(n):
    lists[i] = nx.to_dict_of_lists(graph_list[i])
    n_nodes = n_nodes + graph_list[i].number_of_nodes()

    # Computing the maximum number of nodes in the graphs. It
    # will be used in the computation of vectorial
    # representation.
    if(n_max < graph_list[i].number_of_nodes()):
        n_max = graph_list[i].number_of_nodes()

phi = np.zeros((n_max, n), dtype=np.uint64)

# INITIALIZATION: initialize the nodes labels for each graph
# with their labels or with degrees (for unlabeled graphs)

labels = [0] * n
label_lookup = {}
label_counter = 0

In [ ]:
for i in range(n):
    l_aux = list(nx.get_node_attributes(graph_list[i],'attr_name'))
    # It is assumed that the graph has an attribute
    # 'node_label'
    labels[i] = np.zeros(len(l_aux), dtype=np.int32)

    for j in range(len(l_aux)):
        if not (l_aux[j] in label_lookup):
            label_lookup[l_aux[j]] = label_counter
            labels[i][j] = label_counter
            label_counter += 1
        else:
            labels[i][j] = label_lookup[l_aux[j]]
        # labels are associated to a natural number
        # starting with 0.
        phi[labels[i][j], i] += 1
    #print('graph num: {}'.format(i))

In [ ]:
vectors = np.copy(phi.transpose())

k = np.dot(phi.transpose(), phi)

# MAIN LOOP
it = 0
new_labels = copy.deepcopy(labels)

while it < h:
    # create an empty lookup table
    label_lookup = {}
    label_counter = 0

    phi = np.zeros((n_nodes, n), dtype=np.uint64)
    for i in range(n):
        for v in range(len(lists[i])):
            # form a multiset label of the node v of the i'th graph
            # and convert it to a string

            long_label = np.concatenate((np.array([labels[i][v]]),
                                         np.sort(labels[i]
                                         [lists[i][v]])))
            long_label_string = str(long_label)
            # if the multiset label has not yet occurred, add it to the
            # lookup table and assign a number to it
            if not (long_label_string in label_lookup):
                label_lookup[long_label_string] = label_counter
                new_labels[i][v] = label_counter
                label_counter += 1
            else:
                new_labels[i][v] = label_lookup[long_label_string]
        # fill the column for i'th graph in phi
        aux = np.bincount(new_labels[i])
        try:

            new_row = phi[new_labels[i], i] + aux[new_labels[i]]
            new_row.astype(np.int64)
            phi[new_labels[i], i] = new_row
    

        except:
            print('type: phi' ,phi[new_labels[i], i])
            print('type: aux' ,aux[new_labels[i]])
            raise

    k += np.dot(phi.transpose(), phi)
    labels = copy.deepcopy(new_labels)
    it = it + 1

# Compute the normalized version of the kernel
k_norm = np.zeros(k.shape)
for i in range(k.shape[0]):
    for j in range(k.shape[1]):
        k_norm[i, j] = k[i, j] / np.sqrt(k[i, i] * k[j, j])



In [ ]:
for i in new_labels[:10]:
    print(i)

In [ ]:
test_graph = nx_graphs[0]

In [ ]:
nx.draw_spring(test_graph,with_labels = True,node_color='darkblue',font_color='white')
#plt.savefig('graph_example.png')
#nx.draw(test_sub,with_labels = True)

In [ ]:
from PatchyBuilder import ReceptiveFieldMaker
import sys
sys.path.append('PatchyTools/')
import utils

In [ ]:
w = 18
k = 10
s = 1

In [ ]:
from PatchyBuilder import ReceptiveFieldMaker
def indices_to_one_hot(number, nb_classes,label_dummy=-1):
    """Convert an iterable of indices to one-hot encoded labels."""
    
    if number==label_dummy:
        return np.zeros(nb_classes)
    else:
        return np.eye(nb_classes)[number]

In [ ]:
train = []
for i in range(n):
    rfMaker = ReceptiveFieldMaker(nx_graphs[i], w=w, k=k, s=1, labeling_procedure_name='betweeness',
                              use_node_deg=False, one_hot=7,dummy_value=-1)
    forcnn = rfMaker.make_()
    #f =rfMaker.select_node_sequence()

    train.append(np.array(forcnn).flatten().reshape(k*w,7))

In [ ]:
X_preprocessed = np.array(train)

In [ ]:
print_nodes(test_graph)

In [ ]:
test_graph.add_node(44,gg='gege')


In [ ]:
forcnn = []
for graph in f:
    frelabel = nx.relabel_nodes(graph,nx.get_node_attributes(graph, 'labeling'))  # rename the nodes wrt the labeling
    sorted_idx = sorted(nx.get_node_attributes(frelabel, 'attr').items(), key=lambda x: x[0])
    if rfMaker.one_hot > 0:
        forcnn.append([utils.indices_to_one_hot(x[1], rfMaker.one_hot) for x in sorted_idx])
        
        #forcnn.append(pd.get_dummies(sorted_idx).values)
    else:
        forcnn.append(
            [x[1] for x in sorted_idx])
        #print()

In [ ]:
idx = -5
graph = f[idx]
print_nodes(graph)

In [ ]:
frelabel = nx.relabel_nodes(graph,nx.get_node_attributes(graph, 'labeling'))  # rename the nodes wrt the labeling
sorted_idx = sorted(nx.get_node_attributes(frelabel, 'node_label').items(), key=lambda x: x[0])
print(sorted_idx)

In [ ]:
print_nodes(frelabel)

In [ ]:
print(len(forcnn))
forcnn[idx]

In [ ]:
pd.get_dummies(sorted_idx)

In [ ]:
so

In [ ]:
from PatchyBuilder import 

In [ ]:
test_graph = nx_graphs[0]

In [ ]:
frelabel=nx.relabel_nodes(test_graph,nx.get_node_attributes(test_graph,'node_label'))
#nx.get_node_attributes(test_graph,'node_label')

In [ ]:
for i in frelabel.nodes:
    print(frelabel.node[i])

In [ ]:
def neighborhood_assembly(vertex,nx_graph,k):
    "Output a set of neighbours of the vertex"
    N={vertex}
    L={vertex}
    while len(N)<k and len(L)>0:
        tmp=set()
        for v in L:
            tmp=tmp.union(set(nx_graph.neighbors(v)))
        L=tmp-N
        #print('N: ',N)
        #print(L)
        N=N.union(L)
    #print('last N: ',N)
    return nx_graph.subgraph(list(N))

In [ ]:
def labeling_to_root(graph,vertex):
    labeled_graph=nx.Graph(graph)
    source_path_lengths = nx.single_source_dijkstra_path_length(graph, vertex)
    nx.set_node_attributes(labeled_graph,source_path_lengths,'labeling')

    return labeled_graph


def compute_subgraph_ranking(subgraph,vertex,original_order_to_respect):
    labeled_graph=nx.Graph(subgraph)
    ordered_subgraph_from_centrality=labeling_to_root(subgraph,vertex)

    all_labels_in_subgraph_dict=nx.get_node_attributes(ordered_subgraph_from_centrality,'labeling')

    new_ordered_dict=rank_label_wrt_dict(ordered_subgraph_from_centrality,all_labels_in_subgraph_dict,original_order_to_respect)

    nx.set_node_attributes(labeled_graph,new_ordered_dict,'labeling') 


    return labeled_graph

In [ ]:
ordered_subgraph_from_centrality=labeling_to_root(test_sub,vertex)
all_labels_in_subgraph_dict=nx.get_node_attributes(ordered_subgraph_from_centrality,'labeling')
print_dict(all_labels_in_subgraph_dict)
#new_ordered_dict=rank_label_wrt_dict(ordered_subgraph_from_centrality,all_labels_in_subgraph_dict,original_order_to_respect)

In [ ]:
label_dict = all_labels_in_subgraph_dict
dict_to_respect = original_order_to_respect
all_distinc_labels=list(set(label_dict.values()))
new_ordered_dict=label_dict     

latest_biggest_label=0

for label in all_distinc_labels:
    #print(label)
    nodes_with_this_label = [x for x,y in ordered_subgraph_from_centrality.nodes(data=True) if y['labeling']==label]
    print('nodes ', nodes_with_this_label)
    if len(nodes_with_this_label)>=2:

        inside_ordering=sorted(nodes_with_this_label, key=dict_to_respect.get)
        inside_order_dict=dict(zip(inside_ordering,range(len(inside_ordering))))
    
        for k,v in inside_order_dict.items():

            new_ordered_dict[k]=latest_biggest_label+1+inside_order_dict[k]

        latest_biggest_label=latest_biggest_label+len(nodes_with_this_label)

    else:
        new_ordered_dict[nodes_with_this_label[0]]=latest_biggest_label+1 
        latest_biggest_label=latest_biggest_label+1
    try:
        print('inside: ', inside_ordering)
        print('new: ', new_ordered_dict)
        
    except:

        pass
    print(label,nodes_with_this_label,latest_biggest_label)
    


In [ ]:
print_dict(new_ordered_dict)

In [ ]:
inside_ordering=sorted(nodes_with_this_label, key=dict_to_respect.get)

In [ ]:
for label in all_distinc_labels:

    nodes_with_this_label = [x for x,y in test_sub.nodes(data=True) if y['labeling']==label]
    print(label,nodes_with_this_label)

In [ ]:
subgraph_U=compute_subgraph_ranking(test_sub,vertex,original_order_to_respect)

In [ ]:
print_nodes(test_sub)

In [ ]:
print_nodes(subgraph_U)

In [ ]:
if len(subgraph_U.nodes())>10:

    d=dict(nx.get_node_attributes(subgraph_U,'labeling'))    
    k_first_nodes=sorted(d,key=d.get)[0:k]
    subgraph_N=subgraph_U.subgraph(k_first_nodes)  

    ranked_subgraph_by_labeling_procedure=labeling_procedure(test_sub)['labeled_graph']
    original_order_to_respect=nx.get_node_attributes(ranked_subgraph_by_labeling_procedure,'labeling')        
    subgraph_ranked_N=compute_subgraph_ranking(subgraph_N,vertex,original_order_to_respect)


In [ ]:
def labeling_procedure(graph):   
    
    a=betweenness_centrality_labeling(graph)
    return a

def betweenness_centrality_labeling(graph,approx=None):
    result={}
    #labeled_graph=nx.Graph(graph)
    
    if approx is None:
        centrality=list(nx.betweenness_centrality(graph).items())
    else:
        centrality=list(nx.betweenness_centrality(graph,k=approx).items())
    sorted_centrality=sorted(centrality,key=lambda n:n[1],reverse=True)
    dict_={}
    label=0
    for t in sorted_centrality:
        dict_[t[0]]=label
        label+=1
    nx.set_node_attributes(graph,dict_,'labeling')
    ordered_nodes=list(zip(*sorted_centrality))[0]

    result['labeled_graph']=graph
    result['sorted_centrality']=sorted_centrality
    result['ordered_nodes']=ordered_nodes
    return result

def rank_label_wrt_dict(subgraph,label_dict,dict_to_respect):

    all_distinc_labels=list(set(label_dict.values()))
    new_ordered_dict=label_dict     

    latest_biggest_label=0

    for label in all_distinc_labels:

        nodes_with_this_label = [x for x,y in subgraph.nodes(data=True) if y['labeling']==label]

        if len(nodes_with_this_label)>=2:

            inside_ordering=sorted(nodes_with_this_label, key=dict_to_respect.get)
            inside_order_dict=dict(zip(inside_ordering,range(len(inside_ordering))))

            for k,v in inside_order_dict.items():

                new_ordered_dict[k]=latest_biggest_label+1+inside_order_dict[k]

            latest_biggest_label=latest_biggest_label+len(nodes_with_this_label)

        else :
            new_ordered_dict[nodes_with_this_label[0]]=latest_biggest_label+1 
            latest_biggest_label=latest_biggest_label+1

    return new_ordered_dict

In [ ]:
def normalize_graph(subgraph,vertex):

    "U set of vertices. Return le receptive field du vertex (un graph normalisé)"
    ranked_subgraph_by_labeling_procedure=betweenness_centrality_labeling(subgraph)['labeled_graph']
    original_order_to_respect=nx.get_node_attributes(ranked_subgraph_by_labeling_procedure,'labeling') # à changer je pense
    subgraph_U=compute_subgraph_ranking(subgraph,vertex,original_order_to_respect) #ordonne les noeuds w.r.t labeling procedure

    if len(subgraph_U.nodes())>10:

        d=dict(nx.get_node_attributes(subgraph_U,'labeling'))    
        k_first_nodes=sorted(d,key=d.get)[0:k]
        subgraph_N=subgraph_U.subgraph(k_first_nodes)  

        ranked_subgraph_by_labeling_procedure=labeling_procedure(subgraph)['labeled_graph']
        original_order_to_respect=nx.get_node_attributes(ranked_subgraph_by_labeling_procedure,'labeling')        
        subgraph_ranked_N=compute_subgraph_ranking(subgraph_N,vertex,original_order_to_respect)

    elif len(subgraph_U.nodes())<k:
        subgraph_ranked_N=add_dummy_nodes_at_the_end(subgraph_U)
    else :
        subgraph_ranked_N=subgraph_U

    return canonicalizes(subgraph_ranked_N)

In [ ]:
vertex = 0
test_sub = neighborhood_assembly(vertex,test_graph,10)

In [ ]:
ranked_subgraph_by_labeling_procedure=betweenness_centrality_labeling(test_sub)['labeled_graph']
original_order_to_respect=nx.get_node_attributes(ranked_subgraph_by_labeling_procedure,'labeling') # à changer je pense
#subgraph_U=compute_subgraph_ranking(test_sub,vertex,original_order_to_respect) #ordonne les noeuds w.r.t labeling procedure

In [ ]:
nx.get_node_attributes(ranked_subgraph_by_labeling_procedure,'labeling')

In [ ]:
for i in ranked_subgraph_by_labeling_procedure.nodes:
    print(i,ranked_subgraph_by_labeling_procedure.node[i])

In [ ]:
original_order_to_respect

In [ ]:

centrality=list(nx.betweenness_centrality(test_sub).items())
sorted_centrality = sorted(centrality,key=lambda n:n[1],reverse=True)
sorted_centrality

In [ ]:
dict_={}
label=0
for t in sorted_centrality:
    dict_[t[0]]=label
    label+=1

In [ ]:
import numpy as np
import networkx as nx
import copy


class GK_WL():
    """
    Weisfeiler_Lehman graph kernel.
    """
    def compare_list(self, graph_list, h=1, node_label=True):
        """Compute the all-pairs kernel values for a list of graphs.

        This function can be used to directly compute the kernel
        matrix for a list of graphs. The direct computation of the
        kernel matrix is faster than the computation of all individual
        pairwise kernel values.

        Parameters
        ----------
        graph_list: list
            A list of graphs (list of networkx graphs)
        h : interger
            Number of iterations.
        node_label : boolean
            Whether to use original node labels. True for using node labels
            saved in the attribute 'node_label'. False for using the node
            degree of each node as node attribute.

        Return
        ------
        K: numpy.array, shape = (len(graph_list), len(graph_list))
        The similarity matrix of all graphs in graph_list.

        """
        self.graphs = graph_list
        n = len(graph_list)
        lists = [0] * n
        k = [0] * (h + 1)
        n_nodes = 0
        n_max = 0

        # Compute adjacency lists and n_nodes, the total number of
        # nodes in the dataset.
        for i in range(n):
            lists[i] = graph_list[i].adjacency_list()
            n_nodes = n_nodes + graph_list[i].number_of_nodes()

            # Computing the maximum number of nodes in the graphs. It
            # will be used in the computation of vectorial
            # representation.
            if(n_max < graph_list[i].number_of_nodes()):
                n_max = graph_list[i].number_of_nodes()

        phi = np.zeros((n_max, n), dtype=np.uint64)

        # INITIALIZATION: initialize the nodes labels for each graph
        # with their labels or with degrees (for unlabeled graphs)

        labels = [0] * n
        label_lookup = {}
        label_counter = 0

        # label_lookup is an associative array, which will contain the
        # mapping from multiset labels (strings) to short labels
        # (integers)

        if node_label is True:
            for i in range(n):
                l_aux = nx.get_node_attributes(graph_list[i],
                                               'node_label').values()
                # It is assumed that the graph has an attribute
                # 'node_label'
                labels[i] = np.zeros(len(l_aux), dtype=np.int32)

                for j in range(len(l_aux)):
                    if not (l_aux[j] in label_lookup):
                        label_lookup[l_aux[j]] = label_counter
                        labels[i][j] = label_counter
                        label_counter += 1
                    else:
                        labels[i][j] = label_lookup[l_aux[j]]
                    # labels are associated to a natural number
                    # starting with 0.
                    phi[labels[i][j], i] += 1
        else:
            for i in range(n):
                labels[i] = np.array(graph_list[i].degree().values())
                for j in range(len(labels[i])):
                    phi[labels[i][j], i] += 1

        # Simplified vectorial representation of graphs (just taking
        # the vectors before the kernel iterations), i.e., it is just
        # the original nodes degree.
        self.vectors = np.copy(phi.transpose())

        k = np.dot(phi.transpose(), phi)

        # MAIN LOOP
        it = 0
        new_labels = copy.deepcopy(labels)

        while it < h:
            # create an empty lookup table
            label_lookup = {}
            label_counter = 0

            phi = np.zeros((n_nodes, n), dtype=np.uint64)
            for i in range(n):
                for v in range(len(lists[i])):
                    # form a multiset label of the node v of the i'th graph
                    # and convert it to a string

                    long_label = np.concatenate((np.array([labels[i][v]]),
                                                 np.sort(labels[i]
                                                 [lists[i][v]])))
                    long_label_string = str(long_label)
                    # if the multiset label has not yet occurred, add it to the
                    # lookup table and assign a number to it
                    if not (long_label_string in label_lookup):
                        label_lookup[long_label_string] = label_counter
                        new_labels[i][v] = label_counter
                        label_counter += 1
                    else:
                        new_labels[i][v] = label_lookup[long_label_string]
                # fill the column for i'th graph in phi
                aux = np.bincount(new_labels[i])
                phi[new_labels[i], i] += aux[new_labels[i]]

            k += np.dot(phi.transpose(), phi)
            labels = copy.deepcopy(new_labels)
            it = it + 1

        # Compute the normalized version of the kernel
        k_norm = np.zeros(k.shape)
        for i in range(k.shape[0]):
            for j in range(k.shape[1]):
                k_norm[i, j] = k[i, j] / np.sqrt(k[i, i] * k[j, j])

        return k_norm

    def compare(self, g_1, g_2, h=1, node_label=True):
        """Compute the kernel value (similarity) between two graphs.
        The kernel is normalized to [0,1] by the equation:
        k_norm(g1, g2) = k(g1, g2) / sqrt(k(g1,g1) * k(g2,g2))

        Parameters
        ----------
        g_1 : networkx.Graph
            First graph.
        g_2 : networkx.Graph
            Second graph.
        h : interger
            Number of iterations.
        node_label : boolean
            Whether to use the values under the graph attribute 'node_label'
            as node labels. If False, the degree of the nodes are used as
            labels.

        Returns
        -------
        k : The similarity value between g1 and g2.
        """
        gl = [g_1, g_2]
        return self.compare_list(gl, h, node_label)[0, 1]

In [ ]:
wl = GK_WL()

In [ ]:
wl.compare_list(nx_graphs)

In [ ]:
# Capsule Architecture Parameters:
capsule_params = CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params, conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2, 2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params, caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params, digit_layer_name)

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256  # 250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params, decoder_layer)

# Training Hyperparameters:

args_train = CapsuleTrainingParameters()
data_split = train_test_split(graph_tensor,graph_labels,test_size=0.10,random_state =0)
x_train, x_test, y_train, y_test = data_split
data = ((x_train, y_train), (x_test, y_test))
input_shape = x_train.shape[1:]
n_class = len(np.unique(y_train))
patchy_classifier = GraphClassifier(input_shape,n_class)
patchy_classifier.build_the_graph(capsule_params)